In [ ]:
!pip install jupyterthemes

In [ ]:
!jt -t monokai -f fira -fs 13 -nf ptsans -nfs 10 -N -kl -cursw 5 -cursc r -cellw 80% -T

In [ ]:
#import required libraries

#import pandas library
import pandas as pd
#import numpy
import numpy as np
#import requests
import requests
#import io
import io

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

In [ ]:
#import the dataset in the data variable
url='/content/gdrive/MyDrive/kindle_reviews.csv'
data = pd.read_csv(url)

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
#assigning 50000 rows to data
data=data.head(50000)

In [ ]:
#checking for null values
data.isnull().any()

In [ ]:
data.isnull().sum()

In [ ]:
#deleting or dropping the unwanted columns from the dataset
del data['Unnamed: 0']
del data['asin']
del data['helpful']
del data['reviewTime']
del data['reviewerID']
del data['reviewerName']
del data['unixReviewTime']

In [ ]:
#first 10 rows of data
data.head(10)

In [ ]:
#checking value counts
data.overall.value_counts()

In [ ]:
#check the null values
data.isna().sum()

In [ ]:
#joining review description and summary into one col
data['reviewText']=data['reviewText']+" "+data['summary']

In [ ]:
data.head()

In [ ]:
data.drop(['summary'],axis=1,inplace=True)

In [ ]:
#checking for null values
data.isna().sum()

In [ ]:
#since there is only one null value, replace it with blank space
data['reviewText'].fillna("",inplace = True)

In [ ]:
#Grouping the overall rating of scale 1-5 to 2 categories
def review_sentiment(rating):
    #0(positive) and  with 1(negative) 
    if(rating == 5 or rating == 4 or rating==3):
        return 0
    else:
        return 1

In [ ]:
data.overall = data.overall.apply(review_sentiment)

In [ ]:
data.overall.value_counts()

In [ ]:
data.head(50)

In [ ]:
len(list(data['overall']))

In [ ]:
x=data.iloc[:,1].values

In [ ]:
#import natural language toolkit
import nltk
#nltk.download("stopwords")
#nltk.download("wordnet")
#import stopwords library to remove stopwords
from nltk.corpus import stopwords
#library used for stem the words
from nltk.stem.porter import PorterStemmer
#create an object for stemming
ps = PorterStemmer()
#library used for stem the words
from nltk.stem import WordNetLemmatizer
#create an object for wordnet lemmatizer
wordnet=WordNetLemmatizer()

In [ ]:
import re

In [ ]:
import nltk
nltk.download('all')

In [ ]:
import nltk
nltk.download("stopwords")
nltk.download("wordnet")

In [30]:
 # Initialize empty array to append clean text 
corpus=[]
# no of rows to clean 
for i in range(len(x)):
    #replacing punctuations and numbers using re library 
    temp=re.sub('[^a-zA-Z]',' ',x[i])
    # convert all text to lower cases
    temp=temp.lower()
    # split to array(default delimiter is " ") 
    temp=temp.split()
    # creating WordNetLemmatizer object to take main lemma of each word
    wordnet = WordNetLemmatizer()
    #loop for leammatization each word in string array at ith row 
    temp=[wordnet.lemmatize(word) for word in temp if not word in set(
        stopwords.words('english'))]
    #rejoin all string array elements to create back into a string 
    temp=' '.join(temp)
    #append each string to create array of clean text 
    corpus.append(temp)

In [31]:
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=49b1a2f5c153d3758e6ca30e85d9cbb3d987e7ad5a74ce89b6bd716884fd41b3
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built sklearn


In [32]:
#creating bag of word model
from sklearn.feature_extraction.text import CountVectorizer
#To extract max feature, "max_features" is attribute to 
#experiment with to get better results
cv=CountVectorizer(max_features= 6000)
#z contains vectorized data (independent variable) 
z=cv.fit_transform(corpus).toarray()

In [ ]:
z.shape

In [34]:
#save bag of word model
import joblib
joblib.dump(cv.vocabulary_,"amazo.save")

['amazo.save']

In [35]:
y=data.iloc[:,0].values
y

array([0, 0, 0, ..., 0, 0, 0])

In [36]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(z,y,test_size=0.2,random_state=0)

In [37]:
x_train.shape

(40000, 6000)

In [38]:
x_test.shape

(10000, 6000)

In [39]:
z.shape

(50000, 6000)

In [40]:
#import libraries
import tensorflow
import keras
#import Sequential
from tensorflow.keras.models import Sequential
#import Dense
from tensorflow.keras.layers import Dense

In [ ]:
#intialize the model
model=Sequential()

#adding input layer
model.add(Dense(600,kernel_initializer='random_uniform',
                activation='relu'))

#adding hidden layer
model.add(Dense(100,kernel_initializer='random_uniform',
                activation='relu'))

#adding output layer
model.add(Dense(1,kernel_initializer='random_uniform',
                activation='sigmoid'))

#configure the learning process
model.compile(optimizer='adam',loss='binary_crossentropy',
              metrics=['accuracy'])

In [42]:
print(x_train)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
#training the model
model.fit(x_train,y_train,epochs=20,batch_size=32)

Epoch 1/20
1250/1250 [==============================] - 42s 32ms/step - loss: 0.2031 - accuracy: 0.9240
Epoch 2/20
1250/1250 [==============================] - 40s 32ms/step - loss: 0.1118 - accuracy: 0.9572
Epoch 3/20
1250/1250 [==============================] - 39s 31ms/step - loss: 0.0406 - accuracy: 0.9863
Epoch 4/20
1250/1250 [==============================] - 42s 33ms/step - loss: 0.0076 - accuracy: 0.9977
Epoch 5/20
1250/1250 [==============================] - 39s 31ms/step - loss: 0.0020 - accuracy: 0.9996
Epoch 6/20
1250/1250 [==============================] - 40s 32ms/step - loss: 0.0034 - accuracy: 0.9995
Epoch 7/20
1250/1250 [==============================] - 40s 32ms/step - loss: 2.0488e-04 - accuracy: 1.0000
Epoch 8/20
1250/1250 [==============================] - 41s 33ms/step - loss: 1.4269e-05 - accuracy: 1.0000
Epoch 9/20
1250/1250 [==============================] - 39s 31ms/step - loss: 5.0241e-06 - accuracy: 1.0000
Epoch 10/20
1250/1250 [=============================

In [ ]:
#save the model
model.save("amazo.h5")

In [45]:
ypred=model.predict(x_test)

313/313 [==============================] - 6s 18ms/step


In [46]:
ypred

array([[4.5889658e-38],
       [6.6511050e-30],
       [4.0586614e-28],
       ...,
       [0.0000000e+00],
       [0.0000000e+00],
       [1.3945101e-10]], dtype=float32)

In [47]:
#save bag of word model
import joblib
joblib.dump(cv.vocabulary_,"amazo.save")

['amazo.save']

In [48]:
loaded=CountVectorizer(decode_error='replace',vocabulary=joblib.load('amazo.save'))

In [49]:
d="Writing was good"
d=d.split('delimiter')
result=model.predict(loaded.transform(d))
print(result)
prediction=result>0.5
#print(prediction)
if prediction[0] == False:
    print("Positive review")
elif prediction[0] == True:
    print("Negative review")


1/1 [==============================] - 0s 113ms/step
[[2.5766094e-05]]
Positive review


In [50]:
from tensorflow.keras.models import load_model
model=tensorflow.keras.models.load_model("amazo.h5")

In [51]:
#import load_model function
from tensorflow.keras.models import load_model
#load our saved model file
model=tensorflow.keras.models.load_model("amazo.h5")
#import countvecorizer
from sklearn.feature_extraction.text import CountVectorizer
import joblib
#load saved bag of word model file
loaded=CountVectorizer(decode_error='replace',vocabulary=joblib.load('amazo.save'))

In [52]:
d="good with application"
d=d.split('delimiter')
result=model.predict(loaded.transform(d))
print(result)
prediction=result>0.5
#print(prediction)
if prediction[0] == False:
    print("Positive review")
elif prediction[0] == True:
    print("Negative review")

1/1 [==============================] - 0s 91ms/step
[[1.7416578e-09]]
Positive review
